In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
import plotly.offline as py
import plotly.graph_objs as go
import nltk
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
#dataset link: https://www.kaggle.com/datasets/himanshupoddar/zomato-bangalore-restaurants

In [9]:
zomato_data=pd.read_csv("zomato.csv")
zomato_df=zomato_data.copy()
zomato_df.head(2)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari


In [10]:
zomato_df.shape

(51717, 17)

In [11]:
zomato_df.columns

Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines',
       'approx_cost(for two people)', 'reviews_list', 'menu_item',
       'listed_in(type)', 'listed_in(city)'],
      dtype='object')

In [12]:
zomato_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [13]:
zomato_df.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [14]:
# Drop unnecessary columns
zomato_df = zomato_df.drop(['phone', 'dish_liked', 'url'], axis=1, errors='ignore')

zomato_df.dropna(how='any', inplace=True)

print("Number of duplicates before removal:", zomato_df.duplicated().sum())
zomato_df.drop_duplicates(inplace=True)

zomato_df = zomato_df.rename(columns={
    'approx_cost(for two people)': 'cost',
    'listed_in(type)': 'type',
    'listed_in(city)': 'city'
})

zomato_df = zomato_df[~zomato_df['rate'].isin(['NEW', '-'])].reset_index(drop=True)

zomato_df['rate'] = (
    zomato_df['rate']
    .str.replace('/5', '')  # Remove '/5' if present
    .str.strip()            # Remove whitespace
    .astype(float)          # Convert to float
)

zomato_df['cost'] = (
    zomato_df['cost']
    .astype(str)
    .str.replace(',', '.')  
    .astype(float)
)

Number of duplicates before removal: 34


In [ ]:
# Computing Mean Rating
restaurants = list(zomato_df['name'].unique())
zomato_df['Mean Rating'] = 0.0

for restaurant in restaurants:
    # Calculate mean rating for each restaurant
    mean_rating = zomato_df.loc[zomato_df['name'] == restaurant, 'rate'].mean()
    # Assign the mean rating to all rows for this restaurant
    zomato_df.loc[zomato_df['name'] == restaurant, 'Mean Rating'] = mean_rating

# Scaling the mean rating values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(1, 5))
zomato_df[['Mean Rating']] = scaler.fit_transform(zomato_df[['Mean Rating']]).round(2)

# Checking the mean rating with restaurant name and rating
print(zomato_df[['name', 'rate', 'Mean Rating']].head())

In [ ]:
# Descriptive statistics for numerical columns
print("\nDescriptive statistics for numerical columns:")
print(zomato_df[numerical_cols].describe())

In [ ]:
zomato_df.isnull().sum()

In [ ]:
numerical_cols = zomato_df.select_dtypes(include=np.number).columns.tolist()
print("\nNumerical columns:")
print(numerical_cols)

In [ ]:
# Identify Categorical columns
categorical_cols = zomato_df.select_dtypes(include='object').columns.tolist()
print("\nCategorical columns:")
print(categorical_cols)

In [ ]:
#We will be using the 'Review' and 'Cuisines' feature in order to create a recommender system
## Lower Casing
zomato_df['reviews_list'] = zomato_df['reviews_list'].str.lower()

## Removal of Punctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
zomato_df['reviews_list'] = zomato_df['reviews_list'].apply(lambda text: remove_punctuation(text))

In [ ]:
zomato_df[['reviews_list', 'cuisines']].sample(5)

In [ ]:
print("\n--- Step 4: Outlier Detection (using Box Plots) ---")

plt.figure(figsize=(15, 8))
for i, col in enumerate(numerical_cols):
    plt.subplot(5, 5, i + 1) # Adjust subplot grid based on number of numerical columns
    sns.boxplot(y=zomato_df[col])
    plt.title(f'Box Plot of {col}')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
chains = zomato_df['name'].value_counts()[:6].reset_index()
chains.columns = ['name', 'count']
sns.barplot(data=chains, x='name', y='count', hue='name', dodge=False, palette='tab10', legend=False)
plt.title("Most famous restaurants in Bangalore")
plt.ylabel("Number of outlets")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the figure size
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

# Plot histogram of the 'rate' column
sns.histplot(zomato_df['rate'], bins=20, color='skyblue',edgecolor=None, ax=ax)

# Plot a vertical line for the mean
mean_rating = zomato_df['rate'].mean()
ax.axvline(mean_rating, color='red', linewidth=2, label=f'Mean: {mean_rating:.2f}')

# Add legend, labels, and title
ax.legend()
ax.set_xlabel('Rate', size=20)
ax.set_ylabel('Count', size=20)
ax.set_title('Distribution (count) of Restaurant Rating', size=20)

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Use a font that supports more Unicode characters to avoid glyph warnings
plt.rcParams['font.family'] = 'DejaVu Sans'

# Optional: Clean restaurant names to remove problematic non-ASCII characters
zomato_df['name'] = zomato_df['name'].apply(lambda x: x.encode('ascii', 'ignore').decode())

# Get top 10 rated restaurants
df_rating = zomato_df.drop_duplicates(subset='name')
df_rating = df_rating.sort_values(by='Mean Rating', ascending=False).head(10)

# Set figure size
plt.figure(figsize=(10, 7))

# Barplot
sns.barplot(data=df_rating, x='Mean Rating', y='name', palette='RdBu', legend=False,hue='name')

# Title and axis labels
plt.title('Top Rated 10 Restaurants', fontsize=16)
plt.xlabel('Mean Rating', fontsize=14)
plt.ylabel('Restaurant Name', fontsize=14)

# Improve layout
plt.tight_layout()

# Show plot
plt.show()


In [ ]:
# Sample implementation of get_top_words
def get_top_words(text_series, top_n=15, ngram_range=(2, 2)):
    """Extract top n most frequent n-grams from a pandas Series of text data"""
    vec = CountVectorizer(ngram_range=ngram_range, stop_words='english')
    bag_of_words = vec.fit_transform(text_series.fillna(''))
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:top_n]

# Assuming zomato_df is your DataFrame and 'cuisines' is a column in it
lst = get_top_words(zomato_df['cuisines'], top_n=15, ngram_range=(2, 2))
df_words = pd.DataFrame(lst, columns=['Word', 'Count'])

# Plotting
plt.figure(figsize=(10, 7))
sns.barplot(data=df_words, x='Count', y='Word',hue='Word', palette='Set2',legend=False)  
plt.title('Top 2-Word Cuisine Combinations', fontsize=16)
plt.xlabel('Count', fontsize=14)
plt.ylabel('Cuisine Pair', fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
#numerical cols analysis
numerical_cols = zomato_df.select_dtypes(include=['int64', 'float64']).columns
plt.figure(figsize=(15, 10))  # Bigger, more readable
for i, col in enumerate(numerical_cols):
    plt.subplot((len(numerical_cols) + 2) // 3, 3, i + 1)  # Dynamic rows, 3 columns
    sns.histplot(zomato_df[col], kde=True, color='steelblue', edgecolor='black')  # Styled
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

In [ ]:
#Categorical Analysis
categorical_cols = zomato_df.select_dtypes(include=['object']).columns

def truncate_text(text, max_length=15):
    """Truncate text if it's too long and handle NaN values"""
    if pd.isna(text):
        return 'Unknown'
    text_str = str(text)
    return text_str[:max_length] + '...' if len(text_str) > max_length else text_str
print("\nBar plots for categorical columns:")
if len(categorical_cols) == 0:
    print("No categorical columns found!")
else:
    n_cols = min(3, len(categorical_cols))  # Max 3 columns
    n_rows = (len(categorical_cols) + n_cols - 1) // n_cols
    
    plt.figure(figsize=(15, 4 * n_rows))  # Smaller, more manageable size
    
    for i, col in enumerate(categorical_cols):
        plt.subplot(n_rows, n_cols, i + 1)
        
        # Get top 5 categories
        top_categories = zomato_df[col].value_counts().head(5)
        truncated_categories = [truncate_text(cat, 12) for cat in top_categories.index]
        
        # Create simple bar plot (not seaborn countplot)
        plt.barh(range(len(top_categories)), top_categories.values, color='skyblue')
        plt.yticks(range(len(top_categories)), truncated_categories, fontsize=8)
        plt.xlabel('Count', fontsize=8)
        plt.title(f'Top 5 {col}', fontsize=9)
        plt.gca().tick_params(axis='both', which='major', labelsize=7)
        for j, v in enumerate(top_categories.values):
            plt.text(v + max(top_categories.values) * 0.01, j, str(v), 
                    va='center', fontsize=7)
    
    plt.tight_layout() 
    plt.show()

In [ ]:
#Numerical vs Numerical
print("Numerical vs. Numerical:")
print("\nCorrelation Matrix:")
corr_matrix = zomato_df[numerical_cols].corr()
print(corr_matrix)
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, linewidths=0.5)
plt.title('Correlation matrix of numerical features')
plt.show()

In [ ]:
#Pairplot for scatterplots
print("Pairplot:")
sns.pairplot(zomato_df[numerical_cols].sample(500))
plt.suptitle('Pairplot of Numerical Features',y=1.0) #y is set to maintain the height between the title and plots(vertically)
plt.show()

In [ ]:
#Rating vs online_order-to analyse whether the rating is influenced by the point that online order is offered or not
plt.figure(figsize=(12, 6))
sns.boxplot(x='online_order', y='rate', data=zomato_df)
plt.title('Rating vs Online Order')
plt.show()

In [ ]:
#rating vs city-if restuarant ratings vary accross cities
plt.figure(figsize=(20, 6))
sns.boxplot(x='city', y='rate', data=zomato_df)
plt.title('Rating vs City')
plt.xticks(rotation='vertical')  #making the labels vertical so that the labels don't overlap
plt.tight_layout()
plt.show()

In [ ]:
#rest_type vs cost- How cost to eat varies accross diff types of restuarants
plt.figure(figsize=(20, 6))
sns.boxplot(x='rest_type', y='cost', data=zomato_df)
plt.title('Restaurant type vs Cost')
plt.xticks(rotation='vertical')  #making the labels vertical so that the labels don't overlap
plt.tight_layout()
plt.show()

In [ ]:
#Online Order Availability vs Cost- If restaurants offering online delivery services tend to be cheaper or costlier
plt.figure(figsize=(20, 6))
sns.boxplot(x='online_order', y='cost', data=zomato_df)
plt.title('Online Order Availability vs Cost ')
plt.xticks(rotation='vertical')  #making the labels vertical so that the labels don't overlap
plt.tight_layout()
plt.show()

In [ ]:
#location vs cost
plt.figure(figsize=(20, 6))
sns.boxplot(x='location', y='cost', data=zomato_df)
plt.title('Location(Neighborhood) vs Cost ')
plt.xticks(rotation='vertical')  #making the labels vertical so that the labels don't overlap
plt.tight_layout()
plt.show()

In [ ]:
#online order vs type of meal-online ordering is more common in which type of restaurants 
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.countplot(data=zomato_df, x='type', hue='online_order')
plt.title('Online order availability vs type of meal')
plt.xticks(rotation=45, ha='right')

In [ ]:
plt.figure(figsize=(12, 6))
print("\nCross-tabulation: Table booking option vs location")
rest_type_crosstab= pd.crosstab(zomato_df['book_table'], zomato_df['location'])
print(rest_type_crosstab)
filtered_crosstab = rest_type_crosstab.loc[:, rest_type_crosstab.sum() >= 100]
sns.heatmap(filtered_crosstab, annot=True,annot_kws={"size": 4},fmt='d', cmap='YlGnBu')
plt.title('Heatmap of Table booking option vs location')
plt.tight_layout()
plt.show()

In [ ]:
#only include rows where the rest_type is among the top 10 most common types.
top_rest_types = zomato_df['rest_type'].value_counts().nlargest(10).index
filtered_df = zomato_df[zomato_df['rest_type'].isin(top_rest_types)]
plt.figure(figsize=(12, 6))
sns.countplot(data=filtered_df, x='rest_type', hue='type')
plt.title('restaurant type vs type of meal')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
top_rest_types = zomato_df['rest_type'].value_counts().nlargest(10).index #filtering so that too many rows are not used
filtered_df = zomato_df[zomato_df['rest_type'].isin(top_rest_types)]
print("\nCost by Rest Type, broken down by Online Order:")
plt.figure(figsize=(14, 7))
sns.boxplot(x='rest_type', y='cost', hue='online_order', data=filtered_df)
plt.title('Cost by Rest Type and Online Order Availability')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Online Order')
plt.tight_layout()
plt.show()
print("\nVotes vs Cost, separated by Online Order and colored by Type:")
sns.relplot(
    x='votes', y='cost',
    hue='type', col='online_order',
    data=zomato_df,
    kind='scatter', height=5, aspect=1.2
)
plt.suptitle('Votes vs Cost by Type and Online Order Availability', y=1.05)
plt.tight_layout()
plt.show()

In [ ]:
df_percent = zomato_df[zomato_df['rate'] > 3.5].copy()

# Clean text: fill NaN with empty string and ensure all entries are strings
df_percent.loc[:, 'reviews_list'] = df_percent['reviews_list'].fillna('').astype(str)

# Set index and prepare for similarity calculation
df_percent.set_index('name', inplace=True)
indices = pd.Series(df_percent.index)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])

# Cosine similarity
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors  # This import was missing

def recommend(restaurant_name, top_n=10, df=zomato_df):
    """
    Recommend similar restaurants based on cuisines using TF-IDF and cosine similarity.
    """
    try:
        # 1. Filter dataset (optional memory optimization)
        df_subset = df[df['rate'] > 1.5].copy() if len(df) > 10000 else df.copy()
        
        # 2. Vectorize cuisines using TF-IDF
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = vectorizer.fit_transform(df_subset['cuisines'].fillna(''))
        
        # 3. Fit Nearest Neighbors
        nn = NearestNeighbors(n_neighbors=top_n+1, algorithm='brute', metric='cosine')
        nn.fit(tfidf_matrix)
        
        # 4. Index lookup
        indices = pd.Series(df_subset.index, index=df_subset['name']).drop_duplicates()
        idx = indices.get(restaurant_name)
        
        if idx is None:
            print(f"❌ Restaurant '{restaurant_name}' not found.")
            suggestions = df[df['name'].str.contains(restaurant_name.split()[0], case=False)]['name'].unique()[:5]
            print("Did you mean one of these?\n", suggestions)
            return None
        
        # 5. Get similar restaurants
        distances, neighbor_indices = nn.kneighbors(tfidf_matrix[idx])
        
        # 6. Format and return top N (excluding self)
        recommendations = df_subset.iloc[neighbor_indices[0][1:top_n+1]][
            ['name', 'cuisines', 'rate', 'cost']
        ].sort_values('rate', ascending=False)
        
        recommendations = recommendations.rename(columns={
            'name': 'Restaurant', 
            'cuisines': 'Cuisines',
            'rate': 'Mean Rating',
            'cost': 'Cost'
        })
        
        print(f"\n📍 TOP {top_n} RESTAURANTS LIKE {restaurant_name} WITH SIMILAR REVIEWS:\n")
        return recommendations
        
    except Exception as e:
        print(f"⚠️ Error: {str(e)}")
        print("Try reducing the dataset size or using a machine with more memory.")
        return None

In [ ]:
recommend('Red Chilliez')

In [ ]:
recommend("Cinnamon")

In [ ]:
recommend('Spice Up')

In [ ]:
recommend('Desi Doze')

In [ ]:
import pickle

# After all your preprocessing is done, save everything
with open('restaurant.pkl', 'wb') as file:
    data_to_save = {
        'df_percent': df_percent,
        'tfidf': tfidf,
        'tfidf_matrix': tfidf_matrix,
        'cosine_similarities': cosine_similarities,
        'indices': indices,
        'zomato_df': zomato_df
    }
    pickle.dump(data_to_save, file)
print("✅ Recommendation data saved to restaurant.pkl")